In [3]:
import time

from resume_assistant.application.dataset.upload import upload_to_gcs
from resume_assistant.application.rag.retriever import get_langchain_docs
from resume_assistant.application.rag.utils import extract_content_from_documents
from resume_assistant.application.dataset.extraction import extract_job_description, scrape_webpage_content
from resume_assistant.models.model import get_model
import resume_assistant.application.rag.templates as templates 
import resume_assistant.application.processing.postprocess as postprocess

In [18]:
content = None   # used for local testing
user_resume_path = "ML Engineer Example.pdf"   # file relative directory: ./resume-assistant/data/input/dev/pdfs
url = "https://job-boards.greenhouse.io/scaleai/jobs/4413274005"

### Data ingestion to GCP Cloud storage and indexing

In [19]:
public_url = upload_to_gcs(user_resume_path, content)
documents = get_langchain_docs(user_resume_path)
content = extract_content_from_documents(documents)

root_dir /home/olawale/Desktop/PROJECTS/llms/resume-assistant


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_community import DocAIParser
from langchain_core.document_loaders.blob_loaders import Blob


GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
DOC_PROCESSOR_NAME = os.getenv("DOC_PROCESSOR_NAME")

parser = DocAIParser(location="us",
                    processor_name=DOC_PROCESSOR_NAME,
                    gcs_output_path="gs://{}/resume-assistant/data/output/dev/pdfs".format(GCS_BUCKET_NAME))
inp_path = f"gs://{GCS_BUCKET_NAME}/resume-assistant/data/input/dev/pdfs/{user_resume_path}"
blob = Blob(path=inp_path)
operations = parser.docai_parse([blob])

In [7]:
while parser.is_running(operations):
    time.sleep(0.5)
results = parser.get_results(operations)
docs = list(parser.parse_from_results(results))

### Web/Job page scraping

In [8]:
webpage_content = scrape_webpage_content(url)
job_description = extract_job_description(webpage_content)

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"error":"Forbidden"}\n')


Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"error":"Forbidden"}\n')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"error":"Forbidden"}\n')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"error":"Forbidden"}\n')


### LLM Logic

In [9]:
chat_model = get_model()
ats_chain = templates.ATS_TEMPLATE | chat_model 
ats_result = ats_chain.invoke({"job_description": job_description,
                       "resume_content": content})

In [10]:
json_ats_result = postprocess.get_json_from_result(ats_result)

In [20]:
json_ats_result.keys()

dict_keys(['Summary', 'Experience', 'Skills', 'Missing Keywords', 'Changes Made'])

In [25]:
print(json_ats_result["Experience"][6:])

['2. Machine Learning Engineer, Earth Science Analytics, Athens, Greece (February 2022 - December 2022)', '    - Developed and productionized an automated ML backend service with a Django backend for computer vision workflows using SOTA models like YOLO, Mask RCNN, and ResNet, achieving a 90-95% reduction in manpower hours.', '    - Deployed training and prediction APIs and endpoints for 3D CNN architectures (AttentionUnet, TransUnet, Unet+++, ResUnet) for self-supervised machine learning denoising of seismic images, saving 100% of the time required for manual labeling.', '    - Developed 3D rock property prediction machine learning models using 1D CNN architectures.', '    - Productionized pretrained segmentation and denoising models in EarthNET and EarthVision for clients.', '3. Machine Learning Developer, dGB Earth Sciences, Enschede, Netherlands (March 2021 - February 2022)', '    - Built and integrated a PyTorch machine learning plugin for computer vision workflows into C++ softwa